## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Soltsy,58.1239,30.3238,30.13,94,100,10.07,RU,overcast clouds
1,1,Norman Wells,65.2820,-126.8329,-2.16,84,75,3.44,CA,light snow
2,2,Touros,-5.1989,-35.4608,85.10,62,31,12.41,BR,light rain
3,3,Albany,42.6001,-73.9662,25.81,52,75,1.99,US,broken clouds
4,4,Punta Arenas,-53.1500,-70.9167,60.12,55,29,3.69,CL,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the minimum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the minimum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_city_data_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
filter_city_data_df.head()                                       

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
2,2,Touros,-5.1989,-35.4608,85.10,62,31,12.41,BR,light rain
5,5,Butaritari,3.0707,172.7902,81.36,79,24,13.96,KI,few clouds
7,7,Rikitea,-23.1203,-134.9692,78.08,77,27,19.24,PF,light rain
16,16,Port Augusta,-32.5000,137.7667,78.71,74,72,17.34,AU,broken clouds
21,21,Rio Grande,-32.0350,-52.0986,78.94,54,47,19.60,BR,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
filter_city_data_df.count()

City_ID        217
City           217
Lat            217
Lng            217
Max Temp       217
Humidity       217
Cloudiness     217
Wind Speed     217
Country        215
Description    217
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_city_data_df=filter_city_data_df.dropna()
clean_city_data_df.count()

City_ID        215
City           215
Lat            215
Lng            215
Max Temp       215
Humidity       215
Cloudiness     215
Wind Speed     215
Country        215
Description    215
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_data_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Touros,BR,85.10,light rain,-5.1989,-35.4608,
5,Butaritari,KI,81.36,few clouds,3.0707,172.7902,
7,Rikitea,PF,78.08,light rain,-23.1203,-134.9692,
16,Port Augusta,AU,78.71,broken clouds,-32.5000,137.7667,
21,Rio Grande,BR,78.94,scattered clouds,-32.0350,-52.0986,
26,Atuona,PF,77.99,few clouds,-9.8000,-139.0333,
33,Bengkulu,ID,77.56,overcast clouds,-3.8004,102.2655,
41,Trelew,AR,87.96,clear sky,-43.2490,-65.3051,
45,Oranjestad,AW,82.35,few clouds,12.5240,-70.0270,
46,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
  
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long= row['Lng']
    params['location'] = f'{lat},{long}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'  
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city. 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("",nan_value, inplace=True)
clean_hotel_df=hotel_df.dropna()

clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Touros,BR,85.10,light rain,-5.1989,-35.4608,INN NEW HORIZON
5,Butaritari,KI,81.36,few clouds,3.0707,172.7902,Isles Sunset Lodge
7,Rikitea,PF,78.08,light rain,-23.1203,-134.9692,People ThankYou
16,Port Augusta,AU,78.71,broken clouds,-32.5000,137.7667,Majestic Oasis Apartments
21,Rio Grande,BR,78.94,scattered clouds,-32.0350,-52.0986,Hotel Atlântico Rio Grande
...,...,...,...,...,...,...,...
703,Chimbote,PE,75.61,overcast clouds,-9.0853,-78.5783,Hotel Viera Chimbote
708,Dakar,SN,79.68,scattered clouds,14.6937,-17.4441,Redface Studios Meublés
711,Fort Myers Beach,US,75.25,scattered clouds,26.4520,-81.9481,Lani Kai Island Resort
712,Corrente,BR,83.08,overcast clouds,-3.7367,-43.3394,Chapadinhense Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
city_description=clean_hotel_df['Description']
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))